# CA Gubernatorial and State Elections

This is a new twitter scrape notebook for the California Gubernatorial and other state elections.

### Gubernatorial
The current list of announced and possible candidates that the project will be tracking are the ones listed on _La Times_ article [_California's next governor: Who's running, who's on the fence?_](http://www.latimes.com/politics/la-pol-ca-california-governor-list-2018-htmlstory.html) 

This is a list of the candidates listed in the article as of September 6th, 2017:

**Declared**
* [Gavin Newsom - D](https://twitter.com/GavinNewsom) [02/10/2015](http://www.latimes.com/local/politics/la-me-pol-gavin-newsom-20150212-story.html)
* [John Chiang - D](https://twitter.com/JohnChiangCA) [05/17/2016](http://www.latimes.com/politics/la-pol-sac-essential-poli-john-chiang-jumps-into-californias-2018-governor-1463506797-htmlstory.html)
* [Antonio Villaraigosa - D](https://twitter.com/antonio4ca) [11/10/2016](http://www.dailynews.com/2016/11/10/former-la-mayor-antonio-villaraigosa-launches-bid-for-california-governor/)
* [Delaine Eastin - D](https://twitter.com/DelaineEastin) [11/01/2016](https://ballotpedia.org/Delaine_Eastin)
* [John Cox - R](https://twitter.com/TheRealJohnHCox) [03/07/2017](https://en.wikipedia.org/wiki/John_H._Cox#2018_California_gubernatorial_election)
* [Travis Allen- R](https://twitter.com/JoinTravisAllen) [06/22/2017](https://ballotpedia.org/Travis_Allen)
* [Zoltan Istvan - L](https://twitter.com/zoltan_istvan) [02/12/2017](http://www.newsweek.com/zoltan-istvan-california-governor-libertarian-555088)

**Potential**
* [Kevin Faulconer - R](https://twitter.com/Kevin_Faulconer)
* [Eric Garcetti - D](https://twitter.com/ericgarcetti)
* [Tom Steyer - D](https://twitter.com/TomSteyer)
* [Ashley Sweargin - R](https://twitter.com/ashleycvcf)
* [Steve Westly - D](https://twitter.com/SteveWestly)

### Senate
This is a compiled list from our staff meetings and [Wikipedia](https://en.wikipedia.org/wiki/United_States_Senate_election_in_California,_2018) of likely candidates for the 2018 senate race along with people of interest in CA politics such as Kamala Harris and Jerry Brown.

**Declared**
* [Topher Brennan - D](https://twitter.com/tophertbrennan)
* [Dianne Feinstein - D](https://twitter.com/SenFeinstein)
* [Pat Harris - D](https://twitter.com/PatHarrisCA)
* [David Hildebrand - D](https://twitter.com/David4SenateCA)
* Douglas Howard Pierce - D (no twitter account)
* [John Melendez - D](https://twitter.com/stutteringjohnm)
* [Joe Sanberg - D](https://twitter.com/JosephNSanberg)
* [Steve Stokes - D](https://twitter.com/Stokes4Senate)
* [Kevin de León - D](https://twitter.com/kdeleon)
* Timothy Charles Kalemkarian - R (no twitter account)
* [Caren Lancona - R](https://twitter.com/Carenlancona4Se)
* Stephen James Schrader - R (no twitter account)

**Potential**
* [Eric Garcetti - D](https://twitter.com/ericgarcetti)
* [Loretta Sanchez - D](https://twitter.com/LorettaSanchez)
* [Brad Sherman - D](https://twitter.com/BradSherman)
* [Tom Steyer - D](https://twitter.com/TomSteyer)
* [Eric Swalwell - D](https://twitter.com/RepSwalwell)
* [Kevin Faulconer - R](https://twitter.com/Kevin_Faulconer)
* [Caitlyn Jenner - R](https://twitter.com/Caitlyn_Jenner)
* [Ashley Sweargin - R](https://twitter.com/ashleycvcf)

### People of Interest
This is a list of top California officials and politicians that may be useful for this research. 

**CA Exec**
* [Jerry Brown - D](https://twitter.com/jerrybrowngov)
* [Xavier Becerra - D](https://twitter.com/AGBecerra)
* [Betty Yee - D](https://twitter.com/BettyYeeforCA)
* [Dave Jones - D](https://twitter.com/CA_DaveJones)
* [Tom Torlakson - D](https://twitter.com/TomTorlakson)

**CA Legislature**
* [Toni G. Atkins - D](https://twitter.com/toniatkins)
* [Bill Monning - D](https://twitter.com/billmonning)
* [Jean Fuller - R](https://twitter.com/JeanFuller)
* [Anthony Rendon - D](https://twitter.com/Rendon63rd)
* [Chris Holden - D](https://twitter.com/ChrisHoldenNews)
* [Chad Mayes - R](https://twitter.com/ChadMayesCA)

**CA Senate**
* [Kamala Harris](https://twitter.com/KamalaHarris)

**Party Leadership**
* [Eric Bauman](https://twitter.com/EricBauman)
* Jim Brulte - Inactive

### CA49 Congressional Race
This is a contentious congressional race.

**Democrat**
* [Douglas Applegate](https://twitter.com/ApplegateCA49)
* [Sara Jacobs](https://twitter.com/SaraJacobsCA)
* [Paul Kerr](https://twitter.com/KerrForCongress)
* [Mike Levin](https://twitter.com/MikeLevinCA)
* [Christina Prejean](https://twitter.com/CPforCongress)

**Republican**
* [Rocky Chavez](https://twitter.com/AsmRocky)
* [Kristin Gaspar](https://twitter.com/KristinDGaspar)
* [Diane Harkey](https://twitter.com/DianeHarkey)
* [Brian Maryott](https://twitter.com/brianlmaryott)
* [Joshua Schoonover](https://twitter.com/JSSchoonover)

**Libertarian**
* Joshua Hancock

**Peace and Freedom**
* Jordan Mills

In [2]:
# coding: utf-8

In [4]:
# import necessary python packages
import sys
#sys.path.append("/usr/local/lib/python2.7/site-packages")
import tweepy #https://github.com/tweepy/tweepy
import dropbox #https://www.dropbox.com/developers-v1/core/docs/python
import csv
import time
import os
from datetime import datetime
from collections import defaultdict
import logging
import gspread
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from unidecode import unidecode
import xlsxwriter

#Twitter and Dropbox API credentials
import api_cred as ac

In [5]:
# setup debug logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [6]:
# modify print precison for easier debugging
np.set_printoptions(precision=20)

# Helper Functions

In [8]:
def authenticate_twitter():
  auth = tweepy.OAuthHandler(ac.consumer_key, ac.consumer_secret)
  auth.set_access_token(ac.access_key, ac.access_secret)
  api = tweepy.API(auth)
  return api

In [9]:
def get_new_tweets(tweet_name, since_id):
  api = authenticate_twitter()
  tweets = []
  new_tweets = api.user_timeline(screen_name = tweet_name, since_id = since_id, count = 200)
  tweets.extend(new_tweets)
  if len(tweets) > 0:
    max_id = tweets[-1].id - 1
  while (len(new_tweets) > 0):
    new_tweets = api.user_timeline(screen_name = tweet_name, since_id = since_id, count = 200, max_id = max_id)
    tweets.extend(new_tweets)
    max_id = tweets[-1].id - 1
  
  tweets = [[tweet.id_str, tweet.created_at, tweet.text, "", "", "",tweet.retweet_count, tweet.favorite_count] for tweet in tweets]
  logger.info("Downloading %d tweets from %s" % (len(tweets), tweet_name))
  return tweets[::-1]

In [7]:
def get_lists(df):
  # put twitter handles, last acquired tweet ID, tweet count and store them in respective lists
  names = filter(lambda x: x > 0, df.iloc[:, 1])
  max_ids = df.iloc[:, 2]
  counts = df.iloc[:, 3]
  
  # save the number of entries
  indices = range(1,len(names)+1)
  
  lists = zip(names, max_ids, counts, indices)
  del lists[0] # the first one is column title
  return lists

In [8]:
def load_sheets(path):
  sheet_book = load_workbook(path)
  sheet_writer = pd.ExcelWriter(path, engine='openpyxl')
  sheet_writer.book = sheet_book
  sheet_writer.sheets = dict((ws.title, ws) for ws in sheet_book.worksheets)
  logger.info("Downloaded %s" % path)
  return sheet_writer

In [9]:
# needs to be finished
def illchar(par):
  if (name == 'jimananich'):
    writer = pd.ExcelWriter('id_.xlsx')
    _id = pd.Series(df.iloc[:,0]).astype(str)
    _id.to_excel(writer, sheet_name='id', header=False, index=False, float_format='string')
    writer.save()
    df.to_csv('illchar.csv', encoding='utf-8')

# Write to Sheets ↓

The functions below write data to the currently local sheets.

## Scrape Tweets and save them to respective excel file

*twitter_list.xlsx* contains the list of candidates for each tweets excel file along with the metadata.

*cand_tweets.xlsx* contains the tweets for all announced candidates

*spec_tweets.xlsx* contains the tweets for all the speculated candidates

*rep_tweets.xlsx* contains the tweets for all current CA represenatives of interest for the CA 2018 Elections.

In [10]:
#############################################
# collect_data(tweet_sheet, twitter_list, twitter_sheet)
# This function pulls new latest tweets and appends them to the correct excel file
# params: tweet_sheet - the path to the excel file to save the new tweets 
#         twitter_list - path to file that contains the twitter handles, last tweet pulled id, tweet counts, and 
#                        last pull date
#         twitter_sheet - the correct sheet of accounts corresponding to the tweet sheet passed in
# returns: n/a
def collect_data(tweet_sheet, twitter_list, twitter_sheet):
  # start timer
  start = time.time()
  logger.info("Start...")
  
  # process the paths so they are passable to load_sheets
  tweets_path = os.path.expanduser(tweet_sheet)
  twitter_path = os.path.expanduser(twitter_list)
  
  # load and prepare list of twitter accounts    
  list_writer = load_sheets(twitter_path)
  list_df = pd.read_excel(twitter_path, twitter_sheet)
  
  # list_df['Last_Pulled'] = pd.to_datetime(list_df['Last_Pulled'], errors='coerce') 
  
  # properly load spreadsheet to append new data
  tweet_writer = load_sheets(tweets_path)
  
  # loop through the list of Cand/PACs and updates each tweet sheet appropriately
  for index, row in list_df.iterrows():       
    name, since_id, count = row[1], row[2],row[3]
    
    # check if rep has a twitter handle 
    if (type(name) is not unicode):
      continue
    
    new_tweets = get_new_tweets(name, since_id)
    # if there are no new tweets continue to the next account
    if (len(new_tweets) > 0):
      # turn the new tweets into a dataframe and write them to the corresponding excel sheet
      df = pd.DataFrame(new_tweets)
      
      df.to_excel(tweet_writer, sheet_name=name, startrow=count+1, header=False, index=False, float_format='string')
      
      # update since_id, count, and last_pull date in tweet list
      list_df.iat[index,2] = new_tweets[len(new_tweets)-1][0] # since_id
      list_df.iat[index,3] = count + len(new_tweets) # last_pull
      list_df.iat[index,4] = pd.to_datetime(time.strftime("%m/%d/%Y %H:%M:%S"), errors='coerce') # last_pull date
      
      logger.info("Updated new tweets on spreadsheet for %s" % name)
      time.sleep(20)
  
  # write the updated list and save the changes to the excel sheets
  list_df.Last_ID = list_df.Last_ID.astype(str)
  tweet_writer.save()
  list_df.to_excel(list_writer, sheet_name=twitter_sheet, index=False, float_format='string')
  list_writer.save()
  
  logger.info("Done appending new tweets for %s", twitter_sheet)
  # stop timer and print time elapsed for the current data pull
  end = time.time()
  logger.info("Time Elapsed: %d", float((end-start))/60)

In [11]:
# Purpose: Updates like and retweet totals
def collect_addition_data(tweet_sheet, tweet_list, sheetname):
  # start the timer
  start = time.time()
  logger.info("Start...")
  
  # process the paths so they are passable to load_sheets
  tweet_sheet = os.path.expanduser(tweet_sheet)
  tweet_list = os.path.expanduser(tweet_list)
  
  # load and prepare list of twitter accounts
  list_writer = load_sheets(tweet_list)
  list_df = pd.read_excel(tweet_list, sheetname=sheetname)
  list_df = list_df.dropna(thresh=4)
  # properly load spreadsheet to append new data
  tweet_writer = load_sheets(tweet_sheet)
  logger.info("Downloaded tweets list")
  
  # loop through the list of Cand/PACs and updates each tweet sheet appropriately
  for row in list_df.itertuples():  
    name, since_id, count = row[2], row[3],row[4]
    
    # read cand tweet sheet
    tweets_df = pd.read_excel(tweet_sheet, sheetname=name)
    logger.info("Retrived data from spreadsheet for %s" % name)
    
    # retreive updated tweets
    tweets = get_new_tweets(name, 1)
    updates_df = pd.DataFrame(tweets)
    
    if (updates_df.empty()):
      continue 
      
    # clean dataframe to only include id, retweets, and favorites
    updates_df = updates_df[[0, 6, 7]]
    updates_df.columns = ['id', 'retweets', 'favorites']
    
    # call helper fuction to match updated metadata with correct tweets
    tweets_df = update_metadata(tweets_df, updates_df, name)
    
    if (name == 'antonio4ca'):
      continue
      
    # write the updated data to the twitter profile's sheet to be saved
    tweets_df.to_excel(tweet_writer, sheet_name=name, index=False, startcol=1)
    logger.info("Updated data on spreadsheet for %s" % name)
    # 100 second pause between data pulls to avoid token exceptions
    time.sleep(20)
  
  tweet_writer.save()
  
  logger.info("Done collecting additional data")
  # stop timer and print time elapsed for the current data pull
  end = time.time()
  logger.info("Time Elapsed: %d", float((end-start))/60)

In [12]:
# This function takes the up to date metadata and matches it to their respective tweet using a tweet's unique id
# currently not in use
def update_metadata(tweets_df, updates_df, cand_name): 
  # convert tweet id to the same type as the updates sheet
  tweets_df['id'] = tweets_df['id'].astype(str)
  tweets_df.set_index('id', inplace=True)
  
  ## loop through the updates metadata and updates the tweet sheet
  for row in updates_df.itertuples():
    tweets_df.set_value(row[1], 'retweets', row[2])
    tweets_df.set_value(row[1], 'favorites', row[3])

  # drop null rows that could not match with a tweet
  tweets_df.dropna(subset=['created_at'], inplace=True)
  
  return tweets_df

In [13]:
def create_new_sheets(new_tweet_sheet, new_tweet_list, list_sheet_names):
  # start the timer
  start = time.time()
  logger.info("Start...")
  
  # process the paths so they are passable to load_sheets
  new_tweet_sheet = os.path.expanduser(new_tweet_sheet)
  new_tweet_list = os.path.expanduser(new_tweet_list)
  
  # Create a Pandas Excel writer using XlsxWriter as the engine.
  tweet_writer = pd.ExcelWriter(new_tweet_sheet)
  
  # load and prepare list of twitter accounts
  list_writer = load_sheets(new_tweet_list)
  
  sheet_list = []
  
  for sheetname in list_sheet_names:
    list_df = pd.read_excel(new_tweet_list, sheetname=sheetname)
    list_df = list_df.dropna(axis=0, subset=['Twitter'])
  
    # 
    for row in list_df.itertuples():
      name, since_id, count = row[2], row[3],row[4]
      sheet_list.append(name)
      
      # Create a Pandas dataframe from some data.
      col_headers = {'id': [np.nan], 'created_at': [np.nan], 'text': [np.nan], 'hashtag#': [np.nan], 'at@': [np.nan],
                    'link': [np.nan], 'retweets': [np.nan], 'favorites': [np.nan], 'fullURL': [np.nan]}
      tweets_df = pd.DataFrame(col_headers)
      tweets_df = tweets_df[['id', 'created_at', 'text', 'hashtag#', 'at@', 'link', 'retweets', 'favorites', 'fullURL']]
      
      # write the updated data to the twitter profile's sheet to be saved
      tweets_df.to_excel(tweet_writer, sheet_name=name, index=False, startcol=0)
    
  tweet_writer.save()
  logger.info("Done creating excel doc")
  # stop timer and print time elapsed for the current data pull
  end = time.time()
  logger.info("Time Elapsed: %d", float((end-start))/60)
  return sheet_list

In [14]:
def convert_xlsx_csv (tweet_sheet, sheetname, tweet_list):
  # start timer
  start = time.time()
  logger.info("Start...")
  # dp_client = authenticate_dropbox()
  
  # process the paths so they are passable to load_sheets
  tweet_sheet = os.path.expanduser(tweet_sheet)
  tweet_list = os.path.expanduser(tweet_list)  
    
  # load and prepare list of twitter accounts    
  list_writer = load_sheets(tweet_list)
  list_df = pd.read_excel(tweet_list, sheetname=sheetname)
  #merged_corpus = pd.DataFrame(columns=['id', 'created_at', 'text', 'hashtag#', 'at@', 'link', 'retweets', 'favorites', 'full URL'])
  merged_df = pd.DataFrame()

  initial_loop = True
  
  # loop through the list of Cand/PACs and updates each tweet sheet appropriately
  for index, row in list_df.iterrows():
    name = row[0]
    
    if (initial_loop):
      merged_df = pd.read_excel(tweet_sheet, sheetname=name)
      merged_df['Name'] = name
      num_tweets = len(merged_df.index)
      print num_tweets

      logger.info("Retrived data from spreadsheet for %s" % name)
      initial_loop = False
    
    else:
      # read current cand tweet sheet
      curr_df = pd.read_excel(tweet_sheet, sheetname=name)
      curr_df['Name'] = name
      num_tweets = len(curr_df.index)
      if (num_tweets == 0):
        continue
      
      logger.info("Retrived data from spreadsheet for %s" % name)
      
      merged_df = merged_df.append(curr_df)
  
  print merged_df.shape
  # write the updated list and save the changes to the excel sheets
  #merged_df.to_csv('merged_corpus.csv', encoding='utf-8')
  
  logger.info("done")
  return merged_df

In [15]:
def list_to_number_string(value):
    if isinstance(value, (list, tuple)):
        print str(value) + 'here'
        return str(value)[1:-1]
    else:
        return value

sample_df['text'] = sample_df['text'].apply(list_to_number_string)

NameError: name 'sample_df' is not defined

In [10]:
# func that will pull a number of tweets and create a coding sheet with columns for rating
# sentitment, partisanship, policy, fact, and opinion
def coding_sampling(coding_workbook, sheetname, sample_size, exclude):
  logger.info("Start...")
  start = time.time()
  
  # properly load spreadsheet to append new data
  coding_workbook = os.path.expanduser(coding_workbook)
  sample_writer = load_sheets(coding_workbook)
  full_df = pd.read_excel(coding_workbook, sheetname)
  logger.info("Full data loaded")
  
  col_headers = {'id': [str],'text': [np.nan], 'sentiment': [np.nan], 'political': [np.nan], 'ideology': [np.nan],
                 'macroeconomics': [np.nan], 'national_security': [np.nan], 'crime_law_enforcement': [np.nan],
                 'civil_rights': [np.nan], 'environment': [np.nan], 'education': [np.nan], 'healthcare': [np.nan],
                 'governance': [np.nan], 'no_policy_content': [np.nan], 'asks_for_donation': [np.nan],
                 'ask_to_watch_read_share_follow_something': [np.nan], 'factual_claim': [np.nan], 'opinion': [np.nan]}
  sample_df = pd.DataFrame(col_headers)
  
  # keep only id and text 
  full_df = full_df[['id', 'text']]
  
  # if (exclude):
    # remove already coded tweets
  
  sample_tweets = full_df.sample(sample_size)
  logger.info("Sample made")
  
  sample_df = pd.concat([sample_tweets, sample_df])
  sample_df = sample_df[['id', 'text', 'sentiment', 'political', 'ideology', 'macroeconomics', 'national_security', 
                         'crime_law_enforcement', 'civil_rights', 'environment', 'education', 'healthcare', 'governance',
                         'no_policy_content', 'asks_for_donation', 'ask_to_watch_read_share_follow_something', 
                         'factual_claim', 'opinion']]
  sample_df.drop(sample_df.tail(1).index,inplace=True) # drop last n rows
  logger.info("Sample processed and concated")
  print sample_df.head()
  
  sample_df.id = sample_df.id.astype(str)
  sample_df.to_excel(sample_writer, 'batch1', index=False, float_format='string')
  sample_writer.save()
  
  logger.info("done")
  #return sample_df

In [14]:
coding_sampling(state_data_dir + coded_tweets, all_tweets, coding_sample_size, 0)

INFO:__main__:Start...
INFO:__main__:Downloaded /Users/SoloMune/Dropbox/Summer_of_Tweets/State_Leg_Working_Data/coding_file.xlsx
INFO:__main__:Full data loaded
INFO:__main__:Sample made
INFO:__main__:Sample processed and concated


                        id                                               text  \
335315  974655725854183424  RT @SenatorAOrr: Senate passed $6.6 billion Ed...   
367935  492704480199073792  @AuburnTigers is partnering w/ End Child Hunge...   
552395  542855653589794816  RT @n_cangelosi: Congrats to a former colleagu...   
649013  679699932920569857  RT @b_roe: I try to preach like its someone's ...   
525629  943238323363110913  “The legislation that I have filed for the las...   

        sentiment  political  ideology  macroeconomics  national_security  \
335315        NaN        NaN       NaN             NaN                NaN   
367935        NaN        NaN       NaN             NaN                NaN   
552395        NaN        NaN       NaN             NaN                NaN   
649013        NaN        NaN       NaN             NaN                NaN   
525629        NaN        NaN       NaN             NaN                NaN   

        crime_law_enforcement  civil_rights  envir

INFO:__main__:done


In [ ]:
# reference code on how to save the data to an excel sheet and not lose id data
writer = pd.ExcelWriter('coding_file.xlsx')
merged_sheets.id = merged_sheets.id.astype(str)
merged_sheets.to_excel(writer, 'total_sample', index=False, float_format='string')
writer.save()

#### Tweets Pull Variables

In [12]:
# set file pathway variables an expand to HOME
ca_data_dir = '~/Dropbox/Summer_of_Tweets/ca_working_sheets/'
state_data_dir = '~/Dropbox/Summer_of_Tweets/State_Leg_Working_Data/'

# the excel sheets containing the tweets
cand_tweets = "cand_tweets.xlsx"
spec_tweets = "spec_tweets.xlsx"
rep_tweets = "rep_tweets.xlsx"
state_tweets = "state_tweets.xlsx"
coded_tweets = "coding_file.xlsx"

# the excel file containing the accounts and its sheets
twitter_list = "Twitter_List.xlsx" # this is the name of the metadata lists
cand_sheet = "cand"
spec_sheet = "speculated"
rep_sheet = "reps"
ca49_sheet = "CA49"
state_sheets = ["CA", "AK", "ME", "IL", "GA", "WY", "MI", "IN","HI", "AL", "CT", "WA", "AZ", "FL", "NJ", "OR", "OK",   
                "SD", "WI", "PA"]
all_handles = 'all'
all_tweets = 'total_sample'

coding_sample_size = 1000

In [ ]:
#collect_data(ca_data_dir + cand_tweets, ca_data_dir + twitter_list, cand_sheet)
#print 
#collect_data(ca_data_dir + spec_tweets, ca_data_dir + twitter_list, spec_sheet)
#print 
#collect_data(ca_data_dir + rep_tweets, ca_data_dir + twitter_list, rep_sheet)
#print
#collect_data(ca_data_dir + cand_tweets, ca_data_dir + twitter_list, ca49_sheet)

In [ ]:
collect_addition_data(ca_data_dir + cand_tweets, data_dir + twitter_list, cand_sheet)
print
collect_addition_data(ca_data_dir + spec_tweets, data_dir + twitter_list, spec_sheet)
print
collect_addition_data(ca_data_dir + rep_tweets, data_dir + twitter_list, rep_sheet) # <-- issues here
print
collect_addition_data(ca_data_dir + cand_tweets, data_dir + twitter_list, ca49_sheet)

In [ ]:
for state in state_sheets:
  collect_data(state_data_dir + state_tweets, state_data_dir + twitter_list, state)